In [6]:
import pandas as pd
import requests

In [20]:
bl1_matches = []

for year in range(2010, 2024):
    response = requests.get(f"https://api.openligadb.de/getmatchdata/bl1/{year}")
    years_matches = response.json()
    bl1_matches += years_matches
print(f"Scraped {len(bl1_matches)} bl1 matches")

bl1_tables = {}
for year in range(2010, 2024):
    response = requests.get(f"https://api.openligadb.de/getbltable/bl1/{year}")
    bl1_tables[str(year)] = response.json()

Scraped 4284 bl1 matches


In [21]:
columns = bl1_tables['2018'][0].keys()
columns

dict_keys(['teamInfoId', 'teamName', 'shortName', 'teamIconUrl', 'points', 'opponentGoals', 'goals', 'matches', 'won', 'lost', 'draw', 'goalDiff'])

In [22]:
team_rows = []
for year, teams in bl1_tables.items():
    for team in teams:
        for col in columns:
            team_rows.append([col, year, team['teamInfoId'], team[col]])

df_teams_all = pd.DataFrame(team_rows, columns=['feature', 'year', 'team', 'value'])
print(f"Scraped {len(df_teams_all.team.unique())} teams")

df_teams_all.to_csv("teams.csv")
df_teams_all.head()

Scraped 32 teams


,feature,year,team,value
0,teamInfoId,2010,7,7
1,teamName,2010,7,Borussia Dortmund
2,shortName,2010,7,BVB
3,teamIconUrl,2010,7,https://upload.wikimedia.org/wikipedia/commons...
4,points,2010,7,75


In [18]:
bl1_matches[0]

{'matchID': 27787,
 'matchDateTime': '2014-08-22T20:30:00',
 'timeZoneID': 'W. Europe Standard Time',
 'leagueId': 720,
 'leagueName': '1. Fußball-Bundesliga 2014/2015',
 'leagueSeason': 2014,
 'leagueShortcut': 'bl1',
 'matchDateTimeUTC': '2014-08-22T18:30:00Z',
 'group': {'groupName': '1. Spieltag', 'groupOrderID': 1, 'groupID': 14563},
 'team1': {'teamId': 40,
  'teamName': 'FC Bayern München',
  'shortName': 'Bayern',
  'teamIconUrl': 'https://i.imgur.com/jJEsJrj.png',
  'teamGroupName': None},
 'team2': {'teamId': 131,
  'teamName': 'VfL Wolfsburg',
  'shortName': 'Wolfsburg',
  'teamIconUrl': 'https://i.imgur.com/ucqKV4B.png',
  'teamGroupName': None},
 'lastUpdateDateTime': '2015-05-18T15:44:55.133',
 'matchIsFinished': True,
 'matchResults': [{'resultID': 46215,
   'resultName': 'Endergebnis',
   'pointsTeam1': 2,
   'pointsTeam2': 1,
   'resultOrderID': 1,
   'resultTypeID': 2,
   'resultDescription': 'Ergebnis nach Ende der offiziellen Spielzeit'},
  {'resultID': 46219,
   'r

In [23]:
def match_to_record(row):
    return {
        "id": row['matchID'],
        "date": row['matchDateTimeUTC'],
        "matchDay": row['group']['groupName'],
        "matchDayID": row['group']['groupOrderID'],
        "leagueID": row['leagueId'],
        "leagueName": row['leagueName'],
        "leagueSeason": row['leagueSeason'],
        "team1ID":row['team1']['teamId'],
        "team1Name": row['team1']['teamName'],
        "team1Short":row['team1']['shortName'],
        "team2ID": row['team2']['teamId'],
        "team2Name":row['team2']['teamName'],
        "team2Short": row['team2']['shortName'],
        "team1Goals":row['matchResults'][-1]['pointsTeam1'],
        "team2Goals": row['matchResults'][-1]['pointsTeam2'],
        "result": row['matchResults'][-1]['resultName'],
    }

records = [match_to_record(match) for match in bl1_matches]
df = pd.DataFrame.from_records(records)
df = df.set_index("id")
df.to_csv("matches.csv")
df.tail()

,date,matchDay,matchDayID,leagueID,leagueName,leagueSeason,team1ID,team1Name,team1Short,team2ID,team2Name,team2Short,team1Goals,team2Goals,result
id,,,,,,,,,,,,,,,
66932,2024-05-18T13:30:00Z,34. Spieltag,34,4608,1. Fußball-Bundesliga 2023/2024,2023,131,VfL Wolfsburg,Wolfsburg,81,1. FSV Mainz 05,Mainz,1,3,Endergebnis
66933,2024-05-18T13:30:00Z,34. Spieltag,34,4608,1. Fußball-Bundesliga 2023/2024,2023,175,TSG 1899 Hoffenheim,Hoffenheim,40,FC Bayern München,Bayern,4,2,Endergebnis
66934,2024-05-18T13:30:00Z,34. Spieltag,34,4608,1. Fußball-Bundesliga 2023/2024,2023,134,Werder Bremen,Bremen,129,VfL Bochum,Bochum,4,1,Endergebnis
66935,2024-05-18T13:30:00Z,34. Spieltag,34,4608,1. Fußball-Bundesliga 2023/2024,2023,16,VfB Stuttgart,Stuttgart,87,Borussia Mönchengladbach,Gladbach,4,0,Endergebnis
66936,2024-05-18T13:30:00Z,34. Spieltag,34,4608,1. Fußball-Bundesliga 2023/2024,2023,199,1. FC Heidenheim 1846,Heidenheim,65,1. FC Köln,Köln,4,1,Endergebnis
